In [1]:
!pip install datasets evaluate transformers
!pip install pdfplumber
!pip install --ignore-installed Pillow==9.0.0

  Using cached Pillow-9.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdfplumber 0.9.0 requires Pillow>=9.1, but you have pillow 9.0.0 which is incompatible.


### Imports

In [2]:
import pdfplumber
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForTokenClassification
from transformers import pipeline

import warnings
warnings.filterwarnings('ignore')

## Task 2

In [4]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

In [5]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
text = '''
Jude is an Independent Non- Executive Director of the Bank appointed in 2017. He is the Chairman of the Board and a Banker with over
40 years experience in Banking. Jude graduated from the University of Ghana Business School with a Second Class Upper degree in 1976.
He is an honorary fellow of the Chartered Institute of Bankers Ghana. He commenced his Banking career in Merchant Bank Ghana Limited,
now UMB, in 1978 where he rose to head the Corporate Finance Division of the Bank and became a Director of Merban Investment Holdings Limited,
a subsidiary of the Bank. In 1994, together with a group of investors.'''

ner_results = nlp(text)


In [6]:
ner_results

[{'entity': 'B-PER',
  'score': 0.9990055,
  'index': 1,
  'word': 'Jude',
  'start': 1,
  'end': 5},
 {'entity': 'B-ORG',
  'score': 0.9942794,
  'index': 11,
  'word': 'Bank',
  'start': 55,
  'end': 59},
 {'entity': 'B-PER',
  'score': 0.99883264,
  'index': 36,
  'word': 'Jude',
  'start': 168,
  'end': 172},
 {'entity': 'B-ORG',
  'score': 0.9943738,
  'index': 40,
  'word': 'University',
  'start': 192,
  'end': 202},
 {'entity': 'I-ORG',
  'score': 0.99842584,
  'index': 41,
  'word': 'of',
  'start': 203,
  'end': 205},
 {'entity': 'I-ORG',
  'score': 0.99846375,
  'index': 42,
  'word': 'Ghana',
  'start': 206,
  'end': 211},
 {'entity': 'I-ORG',
  'score': 0.50095254,
  'index': 43,
  'word': 'Business',
  'start': 212,
  'end': 220},
 {'entity': 'I-ORG',
  'score': 0.89762527,
  'index': 44,
  'word': 'School',
  'start': 221,
  'end': 227},
 {'entity': 'B-ORG',
  'score': 0.99903584,
  'index': 61,
  'word': 'Chartered',
  'start': 303,
  'end': 312},
 {'entity': 'I-ORG',
 

## Task 4

In [7]:
def get_page_data(pdf, page_no):
    page = pdf.pages[page_no]
    text = page.extract_text()
    sentences = sent_tokenize(text)
    return text, sentences

In [8]:
pdf_path = "/content/drive/MyDrive/Colab Notebooks/dimensionless/pdfs/chapter.pdf"
pdf = pdfplumber.open(pdf_path)
page_no = 3
text, sentences = get_page_data(pdf, page_no)

In [9]:
text

'GCB Bank PLC Annual Report 2022\nDirectors’\nProfiles\nJude is an Independent Non- Executive Director of the Bank appointed in 2017. He is the\nChairman of the Board and a Banker with over 40 years’ experience in Banking.\nJude graduated from the University of Ghana Business School with a Second Class Upper\ndegree in 1976. He is an honorary fellow of the Chartered Institute of Bankers Ghana. He\ncommenced his Banking career in Merchant Bank Ghana Limited, now UMB, in 1978 where he\nrose to head the Corporate Finance Division of the Bank and became a Director of Merban\nInvestment Holdings Limited, a subsidiary of the Bank.\nIn 1994, together with a group of investors, he set up First Atlantic Merchant Bank Limited\nMr. Jude Kofi Arthur (FAMBL) now First Atlantic Bank and was appointed the first Managing Director of the Bank in\nBoard Chairman 1994. He served as Managing Director until 2012 when he retired honourably.\nHe has had extensive executive education and is associated with su

### Summary

In [10]:
# Next, you need to initialize the tokenizer model:
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

In [11]:
text =  " ".join(sentences[:8])

In [12]:
inputs = tokenizer.encode("summarize: " + text, return_tensors='pt', max_length=512, truncation=True)

summary_ids = model.generate(inputs, max_length=256, min_length=64, length_penalty=5., num_beams=2)
summary = tokenizer.decode(summary_ids[0])

summary

'<pad> Jude graduated from the University of Ghana Business School with a Second Class Upper degree in 1976. he is an honorary fellow of the Chartered Institute of Bankers Ghana. he commenced his Banking career in Merchant Bank Ghana Limited, now UMB, in 1978. in 1994, together with a group of investors, he set up First Atlantic Merchant Bank Limited (FAMBL) now First Atlantic Bank. he served as Managing Director until 2012 when he retired honourably. he has extensive executive education</s>'